In [ ]:
import subprocess
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.table import Table
from astropy.io import fits
from astropy import wcs
from glob import glob
import shlex
from regions import read_ds9, write_ds9
import numpy.ma as ma

# parallel processor et al.
from utilities import parallel_process, check_exe, system_call, system_call_env
from utilities import get_immediate_subdirectories, redshifts_from_papers
from model import inv_beta_model

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
from astropy import log
log.setLevel('WARN')

In [ ]:
# this allows us to run the pipeline not interactively. 
os.environ['HEADASNOQUERY'] = ''
os.environ['HEADASPROMPT'] = '/dev/null'
# os.environ['PFILES'] = '/tmp/$$.tmp/pfiles;$HEADAS/syspfiles'

In [ ]:
def runnh(ra, dec):
    '''This function will automatically run the ftool nh
    on a given RA and Dec in J2000 equinox coordinates'''

    nh = 0
    cmd = f'nh equinox=2000 ra={ra} dec={dec}'
    args = shlex.split(cmd)
    out = ""

    with subprocess.Popen(args,
                          stdout=subprocess.PIPE,
                          universal_newlines=True) as proc:

        proc.wait()
        out = proc.stdout.read()

    outlines = out.split("\n")
    for line in outlines:
        if "Weighted average" in line:
            nh = line.split(" ")[-1]

    return out, float(nh)

In [ ]:
def update_exptime(name, outpath, base_file, regfile, outfile):
    expomap = base_file.replace('cl.evt', 'ex.img')

    # load the data
    expmap_data = fits.getdata(expomap)

    # combine the regions with the annulus.
    coords = read_ds9(regfile, errors='warn')
    m = coords[0].to_mask()
    compound_region = m.to_image(expmap_data.shape)
    final_regmask_inv = np.where(compound_region == 0, 1, 0) # this is the final mask for the data

    expmap_data_masked = ma.masked_array(expmap_data, mask=final_regmask_inv)


    with fits.open(outfile, 'update') as f:
        for hdu in f:
            try:
                hdu.header['EXPOSURE'] = expmap_data_masked.mean()
            except KeyError:
                continue

In [ ]:
def create_bkg_regions(name, outpath):
    ''' For each obs id we are going to create a background region. 10.5 arcmin
    centered on the pointing position of each observation.

    '''

    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt',
                 recursive=True)

    if len(files) < 1:
        return

    # create a place for the products
    if not os.path.isdir(f'{outpath}/{name}/spec_files'):
        os.makedirs(f'{outpath}/{name}/spec_files')

    # get the MCMC fits:
    if os.path.isfile(f'{outpath}/{name}/{name}_mcmcfits.txt'):
        mcmcfit = Table.read(f'{outpath}/{name}/{name}_mcmcfits.txt',
                             format='ascii',
                             header_start=0)
    else:
        return    
    
    # write one background region for each observation
    for f_in in files:

        # get pointing
        point_ra = fits.getheader(f_in, ext=0)['RA_PNT']
        point_dec = fits.getheader(f_in, ext=0)['DEC_PNT']

        # we cannot use the X/Y position in the catalog because it was detected 
        # in the combined frame. We need to use the RA/DEC and then find the 
        # pixel positions in the individual frames. Lame.
        # WCS info to convert world coords to pixels
        hdr = fits.getheader(f_in.replace('cl.evt', 'ex.img'))
        w = wcs.WCS(hdr)
        pixels = w.wcs_world2pix(point_ra, point_dec, 0)

        obs_id = f_in.split('/')[-2]

        # write out the regions
        with open(f'{outpath}/{name}/spec_files/{obs_id}_bkg.reg', 'w') as reg:
            reg.write('image\n')
            # write the background region -- different sizes for different types of obs.
            if f_in[-10] == '3':
                reg.write(
                    f"circle({pixels[0]:.5f},{pixels[1]:.5f},{(11 * 60 /2.36):.5f})\n"
                )
            else:
                reg.write(
                    f"circle({pixels[0]:.5f},{pixels[1]:.5f},{(9.5 * 60 /2.36):.5f})\n"
                )

            for j, xc, yc, rc, rotc in detects[['INDEX', 'RA', 'DEC', 'R', 'ROTANG']]:
                
                # gotta work in pixel coordinates
                pixels = w.wcs_world2pix(xc, yc, 0) 
                
                # we didn't fit all the sources with the MCMC -- use the VTP region
                if j not in mcmcfit['ID']:
                    reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},{rc[0]:.3f}'
                              f',{rc[1]:.3f},{rotc:.3f})\n')
                    continue
                
                # Here we are going to draw a circle with a radius where the beta 
                # model just fades into the background. These values come from 
                # the MCMC fits done previously.
                radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == j],
                                        mcmcfit['rc_50'][mcmcfit['ID'] == j],
                                        mcmcfit['beta_50'][mcmcfit['ID'] == j],
                                        mcmcfit['bg_50'][mcmcfit['ID'] == j])               
                
                reg.write(
                    f"-circle({pixels[0]}, {pixels[1]}, {(1.5 * radius[0] * 60 / 2.36):.3f})\n"
                )
                


In [ ]:
def create_src_regions(name, outpath):
    ''' For each obs id we are going to create a background region. 10.5 arcmin
    centered on the pointing position of each observation.

    '''

    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt',
                 recursive=True)

    if len(files) < 1:
        return

    # create a place for the products
    if not os.path.isdir(f'{outpath}/{name}/spec_files'):
        os.makedirs(f'{outpath}/{name}/spec_files')

    # get the MCMC fits:
    if os.path.isfile(f'{outpath}/{name}/{name}_mcmcfits.txt'):
        mcmcfit = Table.read(f'{outpath}/{name}/{name}_mcmcfits.txt',
                             format='ascii',
                             header_start=0)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_mcmcfits.txt')

    # write one background region for each observation
    for f_in in files:

        obs_id = f_in.split('/')[-2]

        # we cannot use the X/Y position in the catalog because it was detected 
        # in the combined frame. We need to use the RA/DEC and then find the 
        # pixel positions in the individual frames. Lame.
        # WCS info to convert world coords to pixels
        hdr = fits.getheader(f_in.replace('cl.evt', 'ex.img'))
        w = wcs.WCS(hdr)

        # now we have to loop through the detections
        for j, xc, yc, ext in detects[['INDEX', 'RA', 'DEC', 'Extended']]:

            # we didn't fit all the sources with the MCMC
            if j not in mcmcfit['ID']:
                continue
            elif ext < 1:
                continue
                
            # write out the regions
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg',
                      'w') as reg:
                reg.write('image\n')
                
                # Here we are going to draw a circle with a radius where the beta 
                # model just fades into the background. These values come from 
                # the MCMC fits done previously.
                radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == j],
                                        mcmcfit['rc_50'][mcmcfit['ID'] == j],
                                        mcmcfit['beta_50'][mcmcfit['ID'] == j],
                                        mcmcfit['bg_50'][mcmcfit['ID'] == j])

                pixels = w.wcs_world2pix(xc, yc, 0)

                reg.write(
                    f"circle({pixels[0]}, {pixels[1]}, {(radius[0] * 60 / 2.36):.3f})\n"
                )

                # have to loop through them again for the masks
                for k, xc2, yc2, rc, rotc in detects[['INDEX', 'RA', 'DEC', 'R', 'ROTANG']]:
                    
                    if not j == k:
                        
                        pixels = w.wcs_world2pix(xc2, yc2, 0)
                        
                        # we didn't fit all the sources with the MCMC
                        if k not in mcmcfit['ID']:
                            reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},'
                                  f'{rc[0]:.3f},{rc[1]:.3f},{rotc:.3f})\n')
                            continue
                        
                        # Here we are going to draw a circle with a radius where the beta 
                        # model just fades into the background. These values come from 
                        # the MCMC fits done previously.
                        radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == k],
                                                mcmcfit['rc_50'][mcmcfit['ID'] == k],
                                                mcmcfit['beta_50'][mcmcfit['ID'] == k],
                                                mcmcfit['bg_50'][mcmcfit['ID'] == k])

                        reg.write(
                            f"-circle({pixels[0]}, {pixels[1]}, {(1.5 * radius[0] * 60 / 2.36):.3f})\n"
                        )

In [ ]:
def xselect_pha(name, outpath):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt',
                 recursive=True)

    if len(files) < 1:
        return

    # write one background region for each observation
    for f_in in files:

        obs_id = f_in.split('/')[-2]

        # create the background xselect call first
        if not os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_bkg.reg'):
            continue

        # write background xsel.in
        with open(f'{outpath}/{name}/spec_files/{obs_id}_bkg.in', 'w') as f:
            # this is the session name... specify random letters to run many at once.
            f.writelines(f'{obs_id}\n')
            f.writelines('read events\n')
            # set the data directory
            f.writelines('/'.join(f_in.split('/')[:5]) + '\n')
            # first entry
            f.writelines('/'.join(f_in.split('/')[5:]) + '\n')
            f.writelines('yes\n')
            f.writelines(
                f'filter region {outpath}/{name}/spec_files/{obs_id}_bkg.reg\n'
            )
            f.writelines('extract spectrum\n')
            f.writelines(
                f'save spectrum {outpath}/{name}/spec_files/{obs_id}_bkg_pc.pha\n'
            )
            if os.path.isfile(
                    f'{outpath}/{name}/spec_files/{obs_id}_bkg_pc.pha'):
                f.writelines('yes\n')
            f.writelines('exit\n')
            f.writelines('no\n')

            
        # log the output
        log_file = f'{outpath}/{name}/spec_files/{obs_id}_bkg_pha.log'   
        
        # call
        os.system(f'xselect @{outpath}/{name}/spec_files/{obs_id}_bkg.in > {log_file}')
#         stdout, stderr = system_call_env(f'xselect @{outpath}/{name}/spec_files/{obs_id}_bkg.in')
#         with open(log_file, 'w') as f:
#             f.writelines(stdout)

        update_exptime(name, outpath, f_in,
                      f'{outpath}/{name}/spec_files/{obs_id}_bkg.reg',
                      f'{outpath}/{name}/spec_files/{obs_id}_bkg_pc.pha')
            
        # now we do all the sources.
        for j in detects['INDEX']:
            if not os.path.isfile(
                    f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg'):
                continue

            # write source xsel.in
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}.in',
                      'w') as f:
                # this is the session name... specify random letters to run many at once.
                f.writelines(f'{obs_id}_{j}\n')
                f.writelines('read events\n')
                # set the data directory
                f.writelines('/'.join(f_in.split('/')[:5]) + '\n')
                # first entry
                f.writelines('/'.join(f_in.split('/')[5:]) + '\n')
                f.writelines('yes\n')
                f.writelines(
                    f'filter region {outpath}/{name}/spec_files/{obs_id}_{j}.reg\n'
                )
                f.writelines('extract spectrum\n')
                f.writelines(
                    f'save spectrum {outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha\n'
                )
                if os.path.isfile(
                        f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha'):
                    f.writelines('yes\n')
                f.writelines('exit\n')
                f.writelines('no\n')            
            
#             # call
#             stdout, stderr = system_call(f'xselect @{outpath}/{name}/spec_files/{obs_id}_{j}.in')
            
#             # log the output
            log_file = f'{outpath}/{name}/spec_files/{obs_id}_{j}_pha.log'
#             with open(log_file, 'w') as f:
#                 f.writelines(stdout)
                
            os.system(f'xselect @{outpath}/{name}/spec_files/{obs_id}_{j}.in > {log_file}')
        
            update_exptime(name, outpath, f_in,
                           f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg',
                           f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha')


In [ ]:
def mk_arf(name, outpath):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/**xpcw[1-4]po_ex.img',
                 recursive=True)

    if len(files) < 1:
        return

    check_exe('xrtmkarf')

    for f_in in files:

        obs_id = f_in.split('/')[-2]
        # now we have to loop through the detections
        for j, ra, dec in detects[['INDEX', 'RA', 'DEC']]:
            # check that we have the files
            if not os.path.isfile(
                    f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha'):
                continue
            else:
                pha = f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha'

                
            # check to make sure the spectrum has counts    
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pha.log') as logfile:
                outlines = logfile.readlines()
                for line in outlines:
                    line = " ".join(line.split())
                    if "Spectrum has" in line:
                        counts = int(line.split()[2])
                        break

            if not counts > 0:
                continue
                
            # we cannot use the X/Y position in the catalog because it was 
            # detected in the combined frame. We need to use the RA/DEC and then
            # find the pixel positions in the individual frames. Lame.
            # WCS info to convert world coords to pixels
            hdr = fits.getheader(f_in)
            w = wcs.WCS(hdr)
            pixels = w.wcs_world2pix(ra, dec, 0)

            cmd = (f'xrtmkarf phafile={pha} srcx={pixels[0]} srcy={pixels[1]} '
                   f'outfile={outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf '
                   f'psfflag=yes extended=yes expofile={f_in} clobber+')

#             # call
#             stdout, stderr = system_call(cmd)
            
#             # log the output
            log_file = f'{outpath}/{name}/spec_files/{obs_id}_{j}_arf.log'
#             with open(log_file, 'w') as f:
#                 f.writelines(stdout)

            os.system(f'{cmd} > {log_file}')    
                

In [ ]:
def combine_spectra(name, outpath, cnts=10):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # get a list of the reduced obs.
    reduc = get_immediate_subdirectories(f'{outpath}/{name}/reduced')
    reduc_ids = [r.split('/')[-1] for r in reduc]

    if len(reduc) < 1:
        return

    check_exe('addascaspec')
    # we are gonna need to change directories
    wd = os.getcwd()

    # now we do all the sources.
    for j, ext in detects[['INDEX', 'Extended']]:

        if ext < 1:
            continue
        
        # remove files if they are already there.
        if os.path.isfile(f'{outpath}/{name}/spec_files/{j}_pc.pha'):
            os.remove(f'{outpath}/{name}/spec_files/{j}_pc.pha')
        if os.path.isfile(f'{outpath}/{name}/spec_files/{j}_pc.arf'):
            os.remove(f'{outpath}/{name}/spec_files/{j}_pc.arf')
        if os.path.isfile(f'{outpath}/{name}/spec_files/{j}_bkg.pha'):
            os.remove(f'{outpath}/{name}/spec_files/{j}_bkg.pha')
        
        good_reduc_ids = [obs_id for obs_id in reduc_ids 
                         if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg')]

        
        # here we are going to sort the good reduc ids by exposure time.
        # this is because we can't use all of them if there are more than
        # ~15 because addascaspec crashes
        exp_times = []
        for obs_id in good_reduc_ids:
            if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf'):
                exp_times.append(fits.getheader(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha')['EXPOSURE'])
            else:
                exp_times.append(-1.0)
        
        # sorted exp_times -- high to low
        exp_times_argsort = np.argsort(exp_times)[::-1]
        
        # sort good_reduc_ids based on exp_times -- high to low
        good_reduc_ids = [good_reduc_ids[i] for i in exp_times_argsort]
        
        # build the file lists:
        spectra = [f'{obs_id}_{j}_pc.pha' for obs_id in good_reduc_ids
                  if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf')]
        bkgs = [f'{obs_id}_bkg_pc.pha' for obs_id in good_reduc_ids
               if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf')]
        arfs = [f'{obs_id}_{j}_pc.arf' for obs_id in good_reduc_ids
               if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf')]

        # the rmfs are harder to build
        rmfs = []
        for obs_id in good_reduc_ids:
            if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf'):
                with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}_arf.log',
                          'r') as f:
                    lines = f.readlines()
                    for l in lines:
                        # DO NOT REMOVE THE DOUBLE SLASH!!!
                        if '/opt/caldb//data/swift/xrt/cpf/rmf/' in l:
                            rmfs.append(l.split("'")[1])

        if len(spectra) > 0:
        
            # have to write the file list to pass to the spectrum combiner.
            # addascaspec fails when adding too many files together
            with open(f'{outpath}/{name}/spec_files/{j}.list', 'w') as f:
                f.writelines(f'{" ".join(spectra[:20])}\n')
                f.writelines(f'{" ".join(bkgs[:20])}\n')
                f.writelines(f'{" ".join(arfs[:20])}\n')
                f.writelines(f'{" ".join(rmfs[:20])}\n')

            cmd = f'addascaspec {j}.list {j}_pc.pha {j}_pc.arf {j}_bkg.pha'

            # call
            os.chdir(f'{outpath}/{name}/spec_files')
#             stdout, stderr = system_call(cmd)                   
                             
#             # log
            log_file = f'{j}_addascaspec.log'
#             with open(log_file, 'w') as f:
#                 f.writelines(stdout)

            os.system(f'{cmd} > {log_file}')    
                
        # group the counts
        cmd = (f"grppha infile='{j}_pc.pha' outfile='{j}_pc_{cnts}cts.pha' "
               f"chatter=0 comm='bad 1-31 & group min {cnts} & exit' clob+")
        
        if os.path.isfile(f'{j}_pc.pha'):                 
                         
#             # call
#             stdout, stderr = system_call(cmd)                   

#             # log
            log_file = f'{j}_grppha.log'
#             with open(log_file, 'w') as f:
#                 f.writelines(stdout)
            
            os.system(f'{cmd} > {log_file}')
            
            
        os.chdir(wd)
    os.chdir(wd)  

In [ ]:
def mk_xspec_scripts(name, outpath, kT=5, z=0.1, cnts=10):

    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    if not z > 0:
        z = 0.1
        
    # now we have to loop through the detections
    for j, ra, dec in detects[['INDEX', 'RA', 'DEC']]:
        if not os.path.isfile(
                f'{outpath}/{name}/spec_files/{j}_pc_{cnts}cts.pha'):
            continue
        else:
            source_10cts_pha_name = f'{j}_pc_{cnts}cts.pha'

        # get our nH value
        nH = runnh(ra, dec)[1]

        # here is the text we are going to write
        text = [
            "statistic chi\n",
            f"data 1:1 {outpath}/{name}/spec_files/{source_10cts_pha_name}\n",
            f'{outpath}/{name}/spec_files/{j}_pc.arf\n',
            f'{outpath}/{name}/spec_files/{j}_bkg.pha\n',
            "ignore bad\n",
            "ignore 1\n",
            'method leven 10 0.01\n',
            'abund angr\n',
            'xsect bcmc\n',
            'cosmo 70 0 0.70\n',
            'systematic 0\n',
            "model  phabs*mekal\n",
            f"\t{nH/1e22}, -1\n",
            f"\t{kT}, -1\n",
            "\t1\n",
            "\t0.3\n",
            f"\t{z}\n",
            "\t1\n",
            "\t0\n",
            "fit\n",
            f'cpd {outpath}/{name}/spec_files/{j}_pc_{cnts}cts.ps/cps\n',
            'setplot energy\n',
            'plot ldata\n',
            'cpd none\n',
            "error 1.0 7\n",
            "newpar 1 0\n",
            "flux 0.5 2\n",
            "flux 0.1 2.4\n",
            f"lum 0.5 2.0 {z}\n",
            f"lum 0.1 2.4 {z}\n",
            f"data 1:1 {outpath}/{name}/spec_files/{j}_pc.pha\n",
            f'{outpath}/{name}/spec_files/{j}_pc.arf\n',
            f'{outpath}/{name}/spec_files/{j}_bkg.pha\n',
            "ignore bad\n",
            "ignore **-0.5,2.0-**\n",
            "show rates\n",
            "exit\n"
        ]

        #Write it to the script
        with open(f'{outpath}/{name}/spec_files/{j}_pc_{cnts}cts_xspec.in',
                  'w') as script:
            for line in text:
                script.write(line)

In [ ]:
def run_xspec_scripts(name, outpath, cnts=10, rerun=True):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    results = {}
    results['field'] = name
    
    # now we have to loop through the detections
    for j in detects['INDEX']:
        if not os.path.isfile(f'{outpath}/{name}/spec_files/{j}_pc_{cnts}cts_xspec.in'):
            continue
        else:
            source_xspec_in = f'{outpath}/{name}/spec_files/{j}_pc_{cnts}cts_xspec.in'

        # individual source result
        results[j] = {}    
        
        if rerun:
            # build the command and call it
            cmd = f"xspec < {source_xspec_in}"
            stdout, stderr = system_call(cmd, shlexify=False)

            # log
            log_file = f'{outpath}/{name}/spec_files/{j}_xspec.log'
            with open(log_file, 'w') as f:
                f.writelines(stdout)
        
        #Read the fluxes and norms from the stdout
        photon_flux = {}
        energy_flux = {}
        cnt_rate = {}
        norms = []
        norms_err = {}
        chi2 = -1
        DOF = -1
        lum = {}
        
        count = 1
        if rerun:
            outlines = stdout.split('\n')
        else:
            with open(f'{outpath}/{name}/spec_files/{j}_xspec.log', 'r') as f:
                outlines = f.readlines() 
                outlines = [line.strip('\n') for line in outlines]
        for line in outlines:
            # the thaw line adds an additional chi-square output
            # see below
            if "thaw 1" in line:
                count -= 1

            # Here is the actual parsing of the log file
            if "Model Flux" in line:                
                band_low = float(line.split()[7].split("(")[1])
                band_high = float(line.split()[9])
                photons = float(line.split()[2])                
                band = f'{band_low:.1f}-{band_high:.1f}' 
                photon_flux[band] = photons
                energy = float(line.split()[4].split("(")[1])
                energy_flux[band] = energy
                
            elif "2" in line and "mekal" in line and "norm" in line:
                #the norm will be two index before the +/- sign
                pm_index = line.split(" ").index("+/-")
                norms.append(line.split(" ")[pm_index - 2])
#                 norms_err.append(line.split(" ")[pm_index + 2])
            elif "Net count rate" in line:
                pm_index = line.split(" ").index("+/-")
                cnt_rate['Net'] = line.split(" ")[pm_index - 1]
                cnt_rate['Net_err'] = line.split(" ")[pm_index + 1]
            elif "Model predicted rate" in line:
                cnt_rate['Model'] = float(line.split(' ')[4])
            elif "Reduced chi-squared" in line and count <= 2:
                chi2 = float(line.split()[3])
                DOF = int(line.split()[5])
                count += 1
            elif "Model Luminosity" in line:
                band_low = float(line.split()[4].split("(")[1])
                band_high = float(line.split()[6])
                band = f'{band_low:.1f}-{band_high:.1f}'
                lum[band] = float(line.split()[2])
            else:
                pass

        try:
            #Assuming the current script format, this is the one we want
            #This is with PE Absorption
            goodnorm = float(norms[1])

            #Get the index of the line containing the sigmas
            error_index = outlines.index('XSPEC12>error 1.0 7')
            sigma_line = outlines[error_index + 2]
            sigma_low = float(
                sigma_line.split(" ")[-1].split("(")[1].split(",")[0])
            sigma_high = float(
                sigma_line.split(" ")[-1].split("(")[1].split(",")
                [1].split(")")[0])
            sigma_av = (abs(sigma_low) + abs(sigma_high)) / 2

            norms_err['norm_neg'] = abs(sigma_low)
            norms_err['norm_pos'] = sigma_high
                                         
            SN = goodnorm / sigma_av
        except (ValueError, IndexError):
            goodnorm = -1
            SN = -1
            norms_err['norm_neg'] = -1
            norms_err['norm_pos'] = -1

#         # This replaces the commented block above... 
#         # Assuming the current script format, this is the one we want
#         # This is with PE Absorption
#         SN = float(norms[1]) / float(norms_err[1])
        
        results[j]['SN'] = SN
        results[j]['Chi2'] = chi2
        results[j]['DOF'] = DOF
        results[j]['Norm'] = goodnorm
        results[j]['Norm_err'] = norms_err
        results[j]['flux'] = energy_flux
        results[j]['cnt_rate'] = cnt_rate
        results[j]['Lum'] = lum

    results_file = f'{outpath}/{name}/spec_files/{name}_xspec.results'
    with open(results_file, 'w') as f:
        f.write(f'# Spectral Info for {results["field"]}.\n')
        f.write(f'# All fluxes are in the 0.5-2.0 keV band.\n')
        f.write(f'# INDEX SN redCHI2 DOF NORM NORM_NEG NORM_POS '
                'FLUX CNT_RATE_NET CNT_RATE_NET_ERR CNT_RATE_MODEL ECF LUM\n')
        
        try:
            sources = list(results.keys())[1:]
        except AttributeError: # it might not be a dict
            print(f'{name} Error!')
            return results
        
        if len(sources) < 1:
            return results
        
        for indx in sources:
            # basic info first
            f.write(f'{indx} {results[indx]["SN"]:.4f} {results[indx]["Chi2"]:.4f} {results[indx]["DOF"]} ')
            
            # model norms
            f.write(f'{results[indx]["Norm"]} '
                    f'{results[indx]["Norm_err"]["norm_neg"]} '
                    f'{results[indx]["Norm_err"]["norm_pos"]} ')
            
            # write the fluxes
            try:
                flux = results[indx]['flux']['0.5-2.0']
#                 flux = results[indx]['flux']['0.1-2.4']
                f.write(f'{flux} ')
            except KeyError: # no flux or cnt_rate recorded
                flux = -1
                f.write(f'{flux} ')
            
            # write out the count rates
            for cnt_rate_type in ['Net', 'Net_err', 'Model']:
                try:
                    cnt_rate = results[indx]['cnt_rate'][cnt_rate_type]
                    f.write(f'{cnt_rate} ')
                except KeyError:
                    cnt_rate = -1
                    f.write(f'{cnt_rate} ')
            
            # ECF
            if flux < 0:
                f.write(f'{flux} ')
            else:
                try:
                    f.write(f'{flux / results[indx]["cnt_rate"]["Model"]:.4e} ')
                except ZeroDivisionError:
                    f.write('-1 ')

            # Luminosity
            try:
                lum = results[indx]['Lum']['0.5-2.0']
#                 lum = results[indx]['Lum']['0.1-2.4']
                f.write(f'{lum} ')
            except KeyError: # no luminosity
                lum = -1
                f.write(f'{lum} ')    
 
            f.write('\n')

#     print(results)    
    return results

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')

data = redshifts_from_papers(data)

outpath = './data'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath} for n in data['NAME']]
# parallel_process(arr, create_bkg_regions, use_kwargs=True, n_jobs=1)
# parallel_process(arr, create_src_regions, use_kwargs=True, n_jobs=1)
# parallel_process(arr, xselect_pha, use_kwargs=True, n_jobs=4) # 1.5 hours
# parallel_process(arr, mk_arf, use_kwargs=True, n_jobs=4) # 50 minutes
# parallel_process(arr, combine_spectra, use_kwargs=True, n_jobs=1)

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath, 'z':z} for n, z in zip(data['NAME'], data['REDSHIFT'])]
parallel_process(arr, mk_xspec_scripts, use_kwargs=True, n_jobs=1)

# arr = [{'name':n.replace(' ', '_'), 'outpath':outpath, 'rerun':False} for n in data['NAME']]
# results = parallel_process(arr, run_xspec_scripts, use_kwargs=True, n_jobs=1)

In [ ]:
#### For testing ###


outpath ='./data_full'
name = 'PSZ2_G075.08+19.83'
# name = 'PSZ2_G139.62+24.18'
# name = 'PSZ1_G121.35-42.47'
# name = 'PSZ2_G057.80+88.00'

In [ ]:
cleanedresults

In [ ]:
r = cleanedresults[4]

In [ ]:
r

In [ ]:
list(r[1].keys())

In [ ]:
print(f"{0.3 / .0938:.4e}")

In [ ]:
ecf = []
for r in cleanedresults:
    try:
        sources = list(r.keys())[1:]
    except AttributeError: # it's not a dict
        continue
    if len(sources) < 1:
        continue
    for indx in sources:
        try:
            flux = r[indx]['flux']['0.5-2.0']
            cnt_rate = r[indx]['cnt_rate']['Model']
        except KeyError: # no flux or cnt_rate recorded
            continue
        ecf.append(flux/cnt_rate)
        if flux/cnt_rate * 1e11 >3:
            print(r['field'], indx, r[indx], flux/cnt_rate)
#         print(flux/cnt_rate)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
ecf = np.array(ecf)

In [ ]:
plt.hist(ecf * 1e11, bins=25, range=[2,3])
plt.axvline(np.median(ecf) * 1e11)
plt.xlabel('ECF [1e-11]')
plt.xlim(2,3)